### Applying the trained model to test images

Here, the Old City images derived from Street View are evaluated using the trained model from script `0_train_model.ipynb`.

The colab notebook link is: https://colab.research.google.com/drive/1ndJZ7xAZJlPct9nWqIADTT3F0X8cqY0y?usp=sharing

### Setting Up Your Python Environment

In [1]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.6.0+cu124
0.21.0+cu124


In [2]:
%%capture
# Install utility packages
!pip install cjm_pil_utils cjm_pytorch_utils

### Importing the Required Dependencies

In [3]:
# Import Python Standard Library dependencies
import multiprocessing
import os
from pathlib import Path
import random
from typing import Any, Dict, Optional
from functools import partial

# Import utility functions
# from cjm_psl_utils.core import download_file, file_extract, get_source_code
from cjm_pil_utils.core import resize_img, get_img_files, stack_imgs
from cjm_pytorch_utils.core import pil_to_tensor, tensor_to_pil, get_torch_device, set_seed, denorm_img_tensor, move_data_to_device
# from cjm_pandas_utils.core import markdown_to_pandas, convert_to_numeric, convert_to_string
# from cjm_torchvision_tfms.core import ResizeMax, PadSquare, CustomRandomIoUCrop

# Import matplotlib for creating plots
import matplotlib.pyplot as plt

# Import numpy
import numpy as np

# Import the pandas package
import pandas as pd

# Set options for Pandas DataFrame display
pd.set_option('max_colwidth', None)  # Do not truncate the contents of cells in the DataFrame
pd.set_option('display.max_rows', None)  # Display all rows in the DataFrame
pd.set_option('display.max_columns', None)  # Display all columns in the DataFrame

# Import PIL for image manipulation
from PIL import Image, ImageDraw

# Import PyTorch dependencies
import torch
from torch.amp import autocast
from torch.cuda.amp import GradScaler
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
torchvision.disable_beta_transforms_warning()
from torchvision.tv_tensors import BoundingBoxes, Mask
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import torchvision.transforms.v2  as transforms
from torchvision.transforms.v2 import functional as TF

# Import Mask R-CNN
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_V2_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [86]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

### Load trained model and prepare it to evaluate

In [103]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# Load the serialized model parameters
model_loaded = maskrcnn_resnet50_fpn_v2()

# Get the number of input features for the classifier
in_features_box = model_loaded.roi_heads.box_predictor.cls_score.in_features
in_features_mask = model_loaded.roi_heads.mask_predictor.conv5_mask.in_channels

# Get the numbner of output channels for the Mask Predictor
dim_reduced = model_loaded.roi_heads.mask_predictor.conv5_mask.out_channels

# Replace the box predictor
model_loaded.roi_heads.box_predictor = FastRCNNPredictor(in_channels=in_features_box, num_classes=2)

# Replace the mask predictor
model_loaded.roi_heads.mask_predictor = MaskRCNNPredictor(in_channels=in_features_mask, dim_reduced=dim_reduced, num_classes=2)

# Load pre-trained parameters
model_loaded.load_state_dict(torch.load('./drive/MyDrive/MaskRCNN_custom.pth'))

# Set the model to evaluation mode
model_loaded.eval();

# Set device
device = torch.device("cpu")

### Prepare for testing

In [104]:
def get_all_files_with_path(directory):
    file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths

In [117]:
# For each file, run model on image: return True if at least one bench object is found at the 0.9 threshold

def predict_on_images(files):

  results_list = []

  for file in files:
    test_img = Image.open(file).convert('RGB')
    input_img = resize_img(test_img, target_sz=512, divisor=1)

    # Ensure the model and input data are on the same device
    model_loaded.to(device)
    input_tensor = transforms.Compose([transforms.ToImage(), transforms.ToDtype(torch.float32, scale=True)])(input_img)[None].to(device)

    # Make a prediction with the model
    with torch.no_grad():
      model_output = model_loaded(input_tensor)

    # Set the confidence threshold
    threshold = 0.9

    # Specify class names
    class_names = ['background'] + ['bench']

    # Filter the output based on the confidence threshold
    scores_mask = model_output[0]['scores'] > threshold

    # Save and print characteristics of image after prediction
    metric = any(scores_mask)
    results = [file, metric]
    print(results)
    results_list.append(results)

  results_df = pd.DataFrame(results_list, columns=['filename', 'predicted_result']).sort_values(by='filename')

  return results_df

## Perform testing

For the purposes of evaluating model performance, the input images (none of which were part of the training imageset) were screened by the author manually and the images containing valid 'bench' instances were separated. The 'True' and 'False' test images are evaluated below.

In [114]:
dataset_path_true = './drive/MyDrive/street_view_images_true'

files_true = get_all_files_with_path(dataset_path_true)

print(len(files_true))

16


In [111]:
results_true = predict_on_images(files_true)

['./drive/MyDrive/street_view_images_true/file_119.jpg', True]
['./drive/MyDrive/street_view_images_true/file_496.jpg', True]
['./drive/MyDrive/street_view_images_true/file_118.jpg', True]
['./drive/MyDrive/street_view_images_true/file_490.jpg', True]
['./drive/MyDrive/street_view_images_true/file_482.jpg', True]
['./drive/MyDrive/street_view_images_true/file_324.jpg', True]
['./drive/MyDrive/street_view_images_true/file_156.jpg', True]
['./drive/MyDrive/street_view_images_true/file_499.jpg', False]
['./drive/MyDrive/street_view_images_true/file_481.jpg', False]
['./drive/MyDrive/street_view_images_true/file_44.jpg', True]
['./drive/MyDrive/street_view_images_true/file_43.jpg', False]
['./drive/MyDrive/street_view_images_true/file_215.jpg', True]
['./drive/MyDrive/street_view_images_true/file_483.jpg', False]
['./drive/MyDrive/street_view_images_true/file_310.jpg', True]
['./drive/MyDrive/street_view_images_true/file_488.jpg', False]
['./drive/MyDrive/street_view_images_true/file_498.j

In [118]:
dataset_path_false = './drive/MyDrive/street_view_images_false'

files_false = get_all_files_with_path(dataset_path_false)

print(len(files_false))

480


In [119]:
results_false = predict_on_images(files_false)

['./drive/MyDrive/street_view_images_false/file_212.jpg', False]
['./drive/MyDrive/street_view_images_false/file_206.jpg', False]
['./drive/MyDrive/street_view_images_false/file_78.jpg', False]
['./drive/MyDrive/street_view_images_false/file_366.jpg', True]
['./drive/MyDrive/street_view_images_false/file_372.jpg', False]
['./drive/MyDrive/street_view_images_false/file_428.jpg', False]
['./drive/MyDrive/street_view_images_false/file_399.jpg', False]
['./drive/MyDrive/street_view_images_false/file_400.jpg', False]
['./drive/MyDrive/street_view_images_false/file_164.jpg', False]
['./drive/MyDrive/street_view_images_false/file_373.jpg', False]
['./drive/MyDrive/street_view_images_false/file_86.jpg', False]
['./drive/MyDrive/street_view_images_false/file_398.jpg', False]
['./drive/MyDrive/street_view_images_false/file_213.jpg', False]
['./drive/MyDrive/street_view_images_false/file_79.jpg', False]
['./drive/MyDrive/street_view_images_false/file_92.jpg', False]
['./drive/MyDrive/street_view_

Write out results

In [124]:
results_true['actual_value'] = True
results_false['actual_value'] = False

In [125]:
results_concat = pd.concat([results_true, results_false], ignore_index=True)

In [128]:
results_concat.to_csv('./drive/MyDrive/results_concat.csv', index=False)